In [57]:
import pandas as pd
import itertools
import ast

In [2]:
def select_input_id(df, n, k):
    selected_samples = (df
        .groupby(["test_set", "input_id"]).first()
        .groupby("test_set").sample(n*k)
        .groupby("test_set")
    )
    
    indexes = []
    for i in range(0, n*k, n):
        index = [
            list(group.index[i:i+n])
            for _, group in selected_samples            
        ]
        index = list(itertools.chain(*index))
        index = pd.MultiIndex.from_tuples(index)
        indexes.append(index)
        
        
    return indexes

In [25]:
def get_annotator_examples(texts, model, test_set_input_ids):
    texts = texts[texts.index.get_level_values("model").isin(MODELS)]
    examples = []
    for idx in test_set_input_ids:
        exp = texts.xs(idx, level=["test_set", "input_id"], drop_level=False)
        examples.append(exp)
    return pd.concat(examples)

In [3]:
MODELS = [
    (    'llamaVIII', 'lora', 'etrfr'),
    (    'llamaVIII', 'mtllora', 'etrfr+orangesum+wikilarge'),
    ('llamaVIIIinst',     'rag',                     'etrfr'),
    ('llamaVIIIinst',     'rag',           'etrfr+wikilarge')
]
N_SAMPLES = 3
N_ANNOTATORS = 10

In [93]:
texts_df = (
    pd.read_csv("./data/texts.csv", index_col=[0])
    .set_index(["model", "test_set", "input_id"], append=True)
)
texts_df.index = texts_df.index.set_levels(
    [[ast.literal_eval(x) if x[0] == "(" else x for x in texts_df.index.levels[1]],],
    level=[1]
)

In [98]:
texts_df

,,,,inputs,value
,model,test_set,input_id,,
0,"(llamaVIIIinst, rag, etrfr+orangesum+wikilarge)",test_etr_fr_politic,0,"Travailleuses, travailleurs, En avril 2020, p...",Macron a dit en avril 2020 que le pays tenait ...
1,"(llamaVIIIinst, rag, etrfr+orangesum+wikilarge)",test_etr_fr_politic,1,PROFITS EN HAUSSE ET POUVOIR D’ACHAT EN BAISSE...,Les grandes entreprises sont de plus en plus r...
2,"(llamaVIIIinst, rag, etrfr+orangesum+wikilarge)",test_etr_fr_politic,2,AUGMENTATION DES SALAIRES ET DES PENSIONS! Les...,Les travailleurs et les retraités veulent des ...
3,"(llamaVIIIinst, rag, etrfr+orangesum+wikilarge)",test_etr_fr_politic,3,CONTRÔLE DES TRAVAILLEURS SUR LES COMPTES DES ...,Nous devons contrôler les comptes des grandes ...
4,"(llamaVIIIinst, rag, etrfr+orangesum+wikilarge)",test_etr_fr_politic,4,"HÔPITAUX, EHPAD: À BAS LA LOI DU PROFIT! 100 0...",La gestion comptable de la santé publique est ...
...,...,...,...,...,...
1801,"(mistVIIinst, rag, etrfr+orangesum+wikilarge)",test_etr_fr,48,– Tu me permets de rester dormir ici cette nui...,Bou demande à Jules s'il peut dormir chez lui ...
1802,"(mistVIIinst, rag, etrfr+orangesum+wikilarge)",test_etr_fr,49,Un intrus dans la maison – Maow… cela faisait ...,Un intrus a pénétré dans la maison. Bou se rév...
1803,"(mistVIIinst, rag, etrfr+orangesum+wikilarge)",test_etr_fr,50,"– J'ai fini la bouteille qui était au frais, i...",Le père de Jules a fini sa bouteille et demand...


In [95]:
INPUT_IDS = select_input_id(texts_df, N_SAMPLES, N_ANNOTATORS)

In [103]:
assert texts_df[texts_df.index.get_level_values("model").isin(MODELS)].shape[0]/4-53 == 33

In [108]:
get_annotator_examples(texts_df, MODELS, INPUT_IDS[0])

,,,,inputs,value
,model,test_set,input_id,,
665,"(llamaVIII, lora, etrfr)",test_etr_fr,30,"Flairtout aimerait se joindre à eux, mais il n...",Flairtout a peur de l'éléphant. Flairtout foui...
923,"(llamaVIIIinst, rag, etrfr+wikilarge)",test_etr_fr,30,"Flairtout aimerait se joindre à eux, mais il n...",Flairtout veut se joindre à eux mais il est in...
1181,"(llamaVIIIinst, rag, etrfr)",test_etr_fr,30,"Flairtout aimerait se joindre à eux, mais il n...",Flairtout a peur de l'éléphant. Il préfère fou...
1611,"(llamaVIII, mtllora, etrfr+orangesum+wikilarge)",test_etr_fr,30,"Flairtout aimerait se joindre à eux, mais il n...",Flairtout a peur de l'éléphant. Flairtout foui...
657,"(llamaVIII, lora, etrfr)",test_etr_fr,22,Il sort un couteau suisse de sa poche et comme...,Lucas sort un couteau suisse de sa poche. Il c...
915,"(llamaVIIIinst, rag, etrfr+wikilarge)",test_etr_fr,22,Il sort un couteau suisse de sa poche et comme...,Lucas sort un couteau suisse et coupe la corde...
1173,"(llamaVIIIinst, rag, etrfr)",test_etr_fr,22,Il sort un couteau suisse de sa poche et comme...,Lucas sort un couteau suisse de sa poche. Il c...
1603,"(llamaVIII, mtllora, etrfr+orangesum+wikilarge)",test_etr_fr,22,Il sort un couteau suisse de sa poche et comme...,Lucas coupe la corde. « Je te revaudrai ça tou...
644,"(llamaVIII, lora, etrfr)",test_etr_fr,9,Nous atteignons le village au crépuscule. Il e...,Nous arrivons dans le village. Les habitants s...


In [109]:
for i, idx in enumerate(INPUT_IDS):
    df = get_annotator_examples(texts_df, MODELS, idx)
    df.to_csv(f"./data/annot_dfs/samples_ann_{i+1}.csv")